In [ ]:
# !pip3 install 'transformers[torch]'

In [86]:
import json
import os
from googletrans import Translator
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2
import torch
%matplotlib inline


In [28]:
trans = Translator()
json_list = os.listdir('./결과물/json_results')
len(json_list)

4030

In [219]:
total_json = []
for filename in json_list:
    with open('./결과물/json_results/'+filename, 'r') as f:
        data = json.load(f)
        total_json.append(data)


In [165]:
# english name, path dictionary, write as json file
namepath = {}
for whole in total_json:
    l = list(whole['개체명'].split(','))
    ans = []
    for x in l:
        if x[0] == ' ':
            x = x[1:]
        ans.append(enkordict[x].lower())
    namepath[whole['name']] = ans
namepath


 
# Writing to sample.json
with open("./namepath.json", "w", encoding='utf-8') as outfile:
    json.dump(namepath, outfile, ensure_ascii=False, indent=4)
    

In [66]:
total_json[9]['개체명']

'꽃'

In [ ]:
# 사진 띄우기, 옆에 결과 적어놓기, 원래 json파일에 있는 데이터도 적어놓기

In [30]:
object_list = []
for json_dict in total_json:
    object_list += list(json_dict['개체명'].split(','))
object_set = set(object_list)
object_set = list(object_set)

In [31]:
for i, x in enumerate(object_set):
    if x[0] == ' ':
        object_set[i] = x[1:]


In [220]:
enkordict = {}
for kor in object_set:
    enkordict[kor] = trans.translate(kor, dest='en').text
enkordict

In [221]:
labels = [i.lower() for i in enkordict.values()]
clip_labels = [f"an illust of a {label}" for label in labels]
clip_labels

In [222]:
# plot illusts
%matplotlib inline
# fig = plt.figure(figsize=(10, 10))
for i, label_dict in enumerate(total_json[:100]):
    if i % 4 == 0:
        fig = plt.figure(figsize=(10, 10))
    path = './결과물/masking_tape/'+label_dict['name']
    img = Image.open(path)
#     img = cv2.imread(path)
#     fig.add_subplot(rows, columns, i+1)
    fig.add_subplot(1, 4, i%4+1)
    eng_label = ''
    l = list(label_dict['개체명'].split(','))
    for x in l:
        if x[0] == ' ':
            x = x[1:]
        eng_label += enkordict[x] + '/'
    plt.title(eng_label)
    plt.axis('off')
    plt.imshow(img)
# plt.show()



In [223]:
# from torchvision.io import read_image
from torchvision.models import resnet50, ResNet50_Weights


# Step 1: Initialize model with the best available weights
weights = ResNet50_QuantizedWeights.DEFAULT
model = resnet50(weights=weights, quantize=True)
model.eval()

# img = read_image(path)
for i, label_dict in enumerate(total_json[:10]):
    path = './결과물/masking_tape/'+label_dict['name']
    img = torch.tensor(cv2.imread(path))
    img = img.permute(2, 0, 1)


    # Step 2: Initialize the inference transforms
    preprocess = weights.transforms()

    # Step 3: Apply inference preprocessing transforms
    batch = preprocess(img).unsqueeze(0)

    # Step 4: Use the model and print the predicted category
    prediction = model(batch).squeeze(0).softmax(0)
    class_id = prediction.argmax().item()
    
    score = prediction[class_id].item()
    ans = ''
    ind = np.argpartition(prediction.detach().numpy(), -4)[-4:]
    for x in ind:
        ans += weights.meta["categories"][x] + ' '
    category_name = weights.meta["categories"][class_id]
    print(f"{category_name}: {100 * score:.1f}%")
    print(ans)

In [214]:
import torch
from transformers import CLIPProcessor, CLIPModel
import json
# model_id = 'openai/clip-vit-base-patch32'
base_path32 = 'openai/clip-vit-base-patch32'
large_patch14 = "openai/clip-vit-large-patch14"
large_patch14336 = "openai/clip-vit-large-patch14-336"

model_id = large_patch14

In [215]:
# initialize processor, label_tokens, model, device, normalize label embeddings
processor = CLIPProcessor.from_pretrained(model_id)
model = CLIPModel.from_pretrained(model_id)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

label_tokens = processor(text=clip_labels,
                        padding=True,
                        images=None,
                        return_tensors='pt'
                        ).to(device)

label_emb = model.get_text_features(**label_tokens)
label_emb = label_emb.detach().cpu().numpy()
label_emb.shape

#normalize
label_emb = label_emb / np.linalg.norm(label_emb, axis=0)


`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["bos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["eos_token_id"]` will be overriden.


In [224]:

# load image, calculate similarity, retrieve top 3->5 items
# print with original labels

# with open('./namepath.json', 'r') as f:
#     namepath = json.load(f)

pred_list = []; label_list = []
for i, x in enumerate(namepath):
#     if i % 4 == 0:
#         fig = plt.figure(figsize=(10, 10))
    path = './결과물/masking_tape/' + x
    img = Image.open(path)
    image = processor(text=None,
                     images=img,
                     return_tensors='pt'
                     )['pixel_values'].to(device)
    img_emb = model.get_image_features(image)
    img_emb = img_emb.detach().cpu().numpy()
    scores = np.dot(img_emb, label_emb.T)
    pred = np.argsort(scores[0])[::-1][:5]
    top_5 = [labels[x] for x in pred]
#     fig.add_subplot(1, 4, i%4+1)
    pred_out = 'predicted:'+','.join(top_5)
    if 'card' in pred_out or 'cell phone screen' in pred_out:
        pred_out += ', polaroid frame'
    label = 'label:' + ','.join(namepath[x])
    pred_list.append(pred_out)
#     label_list.append(label)
#     label_list.append(namepath[x])
    print(label)
    print(pred_out)
    print()
#     print()
#     plt.title(label+'\n'+pred_out)
#     plt.axis('off')
#     plt.imshow(img)
    

In [213]:
# # base_patch32
# cor = 0
# total = len(label_list)

# for i, x in enumerate(namepath):
#     for item in namepath[x]:
#         if item in pred_list[i]:
#             cor += 1
#             break
# acc = cor/total*100
# print("base_patch32: ", acc)

base_patch32:  56.55086848635236


In [ ]:
# large-patch14
cor = 0
total = len(label_list)

for i, x in enumerate(namepath):
    for item in namepath[x]:
        if item in pred_list[i]:
            cor += 1
            break
acc = cor/total*100
print("large_patch14:", acc)

In [ ]:
# large-patch14-336
cor = 0
total = len(label_list)

for i, x in enumerate(namepath):
    for item in namepath[x]:
        if item in pred_list[i]:
            cor += 1
            break
acc = cor/total*100
print("large_patch14_336:", acc)